## Sparse-promoting D-MORPH based global design optimization

Optimizer: Nelder-Meads<br>
bnd = [[0.625, 1.025], [5e-4, 1.1e-3]]<br>
starting_point = [0.825, 8e-4]

In [ ]:
from scipy.stats import truncnorm
import numpy as np
import chaospy
import numpoly
import scipy.special as sc
import matplotlib.pyplot as plt
import csv
from sklearn.linear_model import LinearRegression 
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LassoLarsCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import Ridge
import kaleido
import random as random
import scipy.linalg

from sklearn.linear_model import Lasso 
from sklearn.linear_model import LassoCV
import warnings
warnings.filterwarnings("ignore")

import pandas as pd

from scipy.optimize import minimize, fmin_l_bfgs_b
import scipy

def clear_variables(exclude_list=None):
    if exclude_list is None:
        exclude_list = []

    exclude_list.append('clear_variables')
    exclude_list.append('__builtins__')
    exclude_list.append('__cached__')
    exclude_list.append('__doc__')
    exclude_list.append('__file__')
    exclude_list.append('__loader__')
    exclude_list.append('__name__')
    exclude_list.append('__package__')
    exclude_list.append('__spec__')


# Average function 
def cal_average(num):
    sum_num = 0
    for t in num:
        sum_num = sum_num + t
        
    avg = sum_num / len(num)
    return avg

# Variance function 
def cal_variance(num):
    avg = cal_average(num)
    sum_num2 = 0
    for t in num:
        sum_num2 = sum_num2 + (t-avg)**2    
    var = (sum_num2/len(num)) 
    return var

# PDD function
def PDD_approx_ex2(N, M, S, df, samples): 

# Total number of basis functions
    LNM = int(sc.binom(N+M,M))
# Number of univariate basis
    LNU = int(1 + N*M)
# Number of bivariate basis
#LNB = LNU + int((N*(N-1)*(M-1)**2)/2)
    LNB = LNU + int(N*(N-1)*M*(M-1)/4)
# Number of trivariate basis
    LNT = 0
    indices = chaospy.glexindex(M+1, dimensions = N)
    inU = np.array(indices[0:LNU,:])
    inB = np.array(indices[0:(LNB),:])
    #inU = []
    cntU = 0
    #Multi-index for univariate (inU)
    for ii in range(LNM):
        cnt = 0
        for jj in range(N):
            if indices[ii,jj] != 0: 
                cnt += 1 
        if cnt == 1: # Univariate
            cntU += 1  
            inU[cntU,:] = indices[ii,:]
        
    cntB, cntU = 0, 0
    ii = 0
    jj = 0
    #Multi-index for bivariate (inB)
    for ii in range(LNM):
        cnt = 0
        for jj in range(N):
            if indices[ii,jj] != 0:
                cnt += 1
        if cnt == 1:
            cntB += 1
            inB[cntB,:] = indices[ii,:]
        if cnt == 2: # Bivariate
            cntB += 1
            inB[cntB,:] = indices[ii,:]

    # generate univariate expansions
    exp1 = chaospy.generate_expansion(M, df['1'], normed=True).T
    exp2 = chaospy.generate_expansion(M, df['2'], normed=True).T
    exp3 = chaospy.generate_expansion(M, df['3'], normed=True).T
    exp4 = chaospy.generate_expansion(M, df['4'], normed=True).T
    exp5 = chaospy.generate_expansion(M, df['5'], normed=True).T

    # get the coefficients of the expansion
    coefficients = [[] for _ in range(N)]
    exponents = [[] for _ in range(N)]
    fcoefficients = [[] for _ in range(N)]
    for ii in range(N):
        crt = ii + 1
        if crt == 1:
            for jj in range(len(exp1)):
                coefficients[ii].append(exp1[jj].coefficients)
                exponents[ii].append(exp1[jj].exponents)
                tfcoefficients = np.zeros(M+1)
                for kk in range(len(coefficients[ii][jj])):
                    tfcoefficients[exp1[jj].exponents[kk]] = exp1[jj].coefficients[kk]
                fcoefficients[ii].append(tfcoefficients)
        if crt == 2:
            for jj in range(len(exp2)):
                coefficients[ii].append(exp2[jj].coefficients)
                exponents[ii].append(exp2[jj].exponents)
                tfcoefficients = np.zeros(M+1)
                for kk in range(len(coefficients[ii][jj])):
                    tfcoefficients[exp2[jj].exponents[kk]] = exp2[jj].coefficients[kk]
                fcoefficients[ii].append(tfcoefficients)
        if crt == 3:
            for jj in range(len(exp3)):
                coefficients[ii].append(exp3[jj].coefficients)
                exponents[ii].append(exp3[jj].exponents)
                tfcoefficients = np.zeros(M+1)
                for kk in range(len(coefficients[ii][jj])):
                    tfcoefficients[exp3[jj].exponents[kk]] = exp3[jj].coefficients[kk]
                fcoefficients[ii].append(tfcoefficients)
        if crt == 4:
            for jj in range(len(exp4)):
                coefficients[ii].append(exp4[jj].coefficients)
                exponents[ii].append(exp4[jj].exponents)
                tfcoefficients = np.zeros(M+1)
                for kk in range(len(coefficients[ii][jj])):
                    tfcoefficients[exp4[jj].exponents[kk]] = exp4[jj].coefficients[kk]
                fcoefficients[ii].append(tfcoefficients)
        if crt == 5:
            for jj in range(len(exp5)):
                coefficients[ii].append(exp5[jj].coefficients)
                exponents[ii].append(exp5[jj].exponents)
                tfcoefficients = np.zeros(M+1)
                for kk in range(len(coefficients[ii][jj])):
                    tfcoefficients[exp5[jj].exponents[kk]] = exp5[jj].coefficients[kk]
                fcoefficients[ii].append(tfcoefficients)

# Generate linear system for regression
# Information matrix
# Initialize information matrix(inM):
    nSample1 = len(samples)
    inM_U = np.zeros((nSample1,LNU))
    inM_B = np.zeros((nSample1,LNB))
    for i in range(LNU):
        for j in range(nSample1):
            inM_U[j][i] = 1
    for i in range(LNB):
        for j in range(nSample1):
            inM_B[j][i] = 1
# generate information matrix(inM):
    if S >= 1:
        for i in range(nSample1):
            # print(f"pdd approx ex2 S1 Progress: {i + 1}/{nSample1}        ", end="\r")
            for j in range(LNU):
                tmp = 1
                for k in range(N):
                        crt = int(inU[j][k])
                        tmp *= np.polynomial.polynomial.polyval(samples[i,k], fcoefficients[k][crt])
                        if crt != 0:
                            inM_U[i][j] = tmp
# generate information matrix(inM_B):
    if S >= 2:
        for i in range(nSample1):
            # print(f"pdd approx ex2 S2 Progress: {i + 1}/{nSample1}         ", end="\r")
            for j in range(LNB):
                tmp = 1
                for k in range(N):
                       crt = int(inB[j][k])
                       tmp *= np.polynomial.polynomial.polyval(samples[i,k], fcoefficients[k][crt])
                       if crt != 0:
                           inM_B[i][j] = tmp
    if S==1:
        return np.array(inM_U), LNU, inU
    if S==2:
        return np.array(inM_B), LNB, inB 
    
def Generate_expansion(M):

    # generate univariate expansions
    # bases consistent with Z
    exp1 = chaospy.generate_expansion(M, chaospy.Uniform(a1, b1), normed=True).T
    exp2 = chaospy.generate_expansion(M, chaospy.TruncNormal(a2/mu2, b2/mu2, 1, sigma2/mu2), normed=True).T
    exp3 = chaospy.generate_expansion(M, chaospy.Uniform(a3/8e-4, b3/8e-4), normed=True).T
    exp4 = chaospy.generate_expansion(M, chaospy.Uniform(a4, b4), normed=True).T
    exp5 = chaospy.generate_expansion(M, chaospy.TruncNormal(a5, b5, mu5, sigma5), normed=True).T

    # get the coefficients of the expansion
    coefficients = [[] for _ in range(N)]
    exponents = [[] for _ in range(N)]
    fcoefficients = [[] for _ in range(N)]
    for ii in range(N):
        crt = ii + 1
        if crt == 1:
            for jj in range(len(exp1)):
                coefficients[ii].append(exp1[jj].coefficients)
                exponents[ii].append(exp1[jj].exponents)
                tfcoefficients = np.zeros(M+1)
                for kk in range(len(coefficients[ii][jj])):
                    tfcoefficients[exp1[jj].exponents[kk]] = exp1[jj].coefficients[kk]
                fcoefficients[ii].append(tfcoefficients)
        if crt == 2:
            for jj in range(len(exp2)):
                coefficients[ii].append(exp2[jj].coefficients)
                exponents[ii].append(exp2[jj].exponents)
                tfcoefficients = np.zeros(M+1)
                for kk in range(len(coefficients[ii][jj])):
                    tfcoefficients[exp2[jj].exponents[kk]] = exp2[jj].coefficients[kk]
                fcoefficients[ii].append(tfcoefficients)
        if crt == 3:
            for jj in range(len(exp3)):
                coefficients[ii].append(exp3[jj].coefficients)
                exponents[ii].append(exp3[jj].exponents)
                tfcoefficients = np.zeros(M+1)
                for kk in range(len(coefficients[ii][jj])):
                    tfcoefficients[exp3[jj].exponents[kk]] = exp3[jj].coefficients[kk]
                fcoefficients[ii].append(tfcoefficients)
        if crt == 4:
            for jj in range(len(exp4)):
                coefficients[ii].append(exp4[jj].coefficients)
                exponents[ii].append(exp4[jj].exponents)
                tfcoefficients = np.zeros(M+1)
                for kk in range(len(coefficients[ii][jj])):
                    tfcoefficients[exp4[jj].exponents[kk]] = exp4[jj].coefficients[kk]
                fcoefficients[ii].append(tfcoefficients)
        if crt == 5:
            for jj in range(len(exp5)):
                coefficients[ii].append(exp5[jj].coefficients)
                exponents[ii].append(exp5[jj].exponents)
                tfcoefficients = np.zeros(M+1)
                for kk in range(len(coefficients[ii][jj])):
                    tfcoefficients[exp5[jj].exponents[kk]] = exp5[jj].coefficients[kk]
                fcoefficients[ii].append(tfcoefficients)
                
    return fcoefficients

def Cal_infoM(samples, N, M, dv_0, dv, fcoefficients):

    dv_1_0 = dv_0[0]
    dv_2_0 = dv_0[1]

    dv_1 = dv[0]
    dv_2 = dv[1]

    samples_transformed = np.empty((samples.shape))

    samples_transformed[:, 0] = samples[:, 0]
    samples_transformed[:, 1] = samples[:, 1] * dv_1/dv_1_0
    samples_transformed[:, 2] = samples[:, 2] * dv_2/dv_2_0
    samples_transformed[:, 3] = samples[:, 3]
    samples_transformed[:, 4] = samples[:, 4]

    # Total number of basis functions
    LNM = int(sc.binom(N+M, M))
    # Number of univariate basis
    LNU = int(1 + N*M)
    # Number of bivariate basis
    LNB = LNU + int(N*(N-1)*M*(M-1)/4)

    indices = chaospy.glexindex(M+1, dimensions = N)

    inU = np.array(indices[0:LNU,:])
    inB = np.array(indices[0:(LNB),:])
    #inU = []
    cntU = 0
    #Multi-index for univariate (inU)
    for ii in range(LNM):
        cnt = 0
        for jj in range(N):
            if indices[ii,jj] != 0: 
                cnt += 1 
        if cnt == 1: # Univariate
            cntU += 1  
            inU[cntU,:] = indices[ii,:]
        
    cntB, cntU = 0, 0
    ii = 0
    jj = 0
    #Multi-index for bivariate (inB)
    for ii in range(LNM):
        cnt = 0
        for jj in range(N):
            if indices[ii,jj] != 0:
                cnt += 1
        if cnt == 1:
            cntB += 1
            inB[cntB,:] = indices[ii,:]
        if cnt == 2: # Bivariate
            cntB += 1
            inB[cntB,:] = indices[ii,:]

    # Generate linear system for regression
    # Information matrix
    # Initialize information matrix(inM):
    nSample1 = len(samples)
    inM = np.ones((nSample1, LNB))

    for i in range(nSample1):
        if i % 100 == 0:
            print(f"infoM Progress: {i + 1}/{nSample1}", end="\r")
        for j in range(LNB):
            tmp = 1
            for k in range(N):
                crt = int(inB[j][k])
                tmp *= np.polynomial.polynomial.polyval(samples_transformed[i,k], fcoefficients[k][crt])
                if crt != 0:
                    inM[i][j] = tmp

    return inM, inB, LNB

def Generate_expansion_Q(M, df):

    # generate univariate expansions, for predicting Q using X, (X = Z*mu)
    # expansion should be consistent with X
    exp1 = chaospy.generate_expansion(M, df['1'], normed=True).T
    exp2 = chaospy.generate_expansion(M, df['2'], normed=True).T
    exp3 = chaospy.generate_expansion(M, df['3'], normed=True).T
    exp4 = chaospy.generate_expansion(M, df['4'], normed=True).T
    exp5 = chaospy.generate_expansion(M, df['5'], normed=True).T

    # get the coefficients of the expansion
    coefficients = [[] for _ in range(N)]
    exponents = [[] for _ in range(N)]
    fcoefficients = [[] for _ in range(N)]
    for ii in range(N):
        crt = ii + 1
        if crt == 1:
            for jj in range(len(exp1)):
                coefficients[ii].append(exp1[jj].coefficients)
                exponents[ii].append(exp1[jj].exponents)
                tfcoefficients = np.zeros(M+1)
                for kk in range(len(coefficients[ii][jj])):
                    tfcoefficients[exp1[jj].exponents[kk]] = exp1[jj].coefficients[kk]
                fcoefficients[ii].append(tfcoefficients)
        if crt == 2:
            for jj in range(len(exp2)):
                coefficients[ii].append(exp2[jj].coefficients)
                exponents[ii].append(exp2[jj].exponents)
                tfcoefficients = np.zeros(M+1)
                for kk in range(len(coefficients[ii][jj])):
                    tfcoefficients[exp2[jj].exponents[kk]] = exp2[jj].coefficients[kk]
                fcoefficients[ii].append(tfcoefficients)
        if crt == 3:
            for jj in range(len(exp3)):
                coefficients[ii].append(exp3[jj].coefficients)
                exponents[ii].append(exp3[jj].exponents)
                tfcoefficients = np.zeros(M+1)
                for kk in range(len(coefficients[ii][jj])):
                    tfcoefficients[exp3[jj].exponents[kk]] = exp3[jj].coefficients[kk]
                fcoefficients[ii].append(tfcoefficients)
        if crt == 4:
            for jj in range(len(exp4)):
                coefficients[ii].append(exp4[jj].coefficients)
                exponents[ii].append(exp4[jj].exponents)
                tfcoefficients = np.zeros(M+1)
                for kk in range(len(coefficients[ii][jj])):
                    tfcoefficients[exp4[jj].exponents[kk]] = exp4[jj].coefficients[kk]
                fcoefficients[ii].append(tfcoefficients)
        if crt == 5:
            for jj in range(len(exp5)):
                coefficients[ii].append(exp5[jj].coefficients)
                exponents[ii].append(exp5[jj].exponents)
                tfcoefficients = np.zeros(M+1)
                for kk in range(len(coefficients[ii][jj])):
                    tfcoefficients[exp5[jj].exponents[kk]] = exp5[jj].coefficients[kk]
                fcoefficients[ii].append(tfcoefficients)
                
    return fcoefficients

def Cal_Q(samples, N, M, dv, fcoefficients_Q):

    mu_2 = dv[0]
    mu_3 = dv[1]

    samples_transformed = np.empty((samples.shape))

    samples_transformed[:, 0] = samples[:, 0] 
    samples_transformed[:, 1] = samples[:, 1] * mu_2
    samples_transformed[:, 2] = samples[:, 2] * mu_3
    samples_transformed[:, 3] = samples[:, 3] 
    samples_transformed[:, 4] = samples[:, 4] 

    # Total number of basis functions
    LNM = int(sc.binom(N+M, M))
    # Number of univariate basis
    LNU = int(1 + N*M)
    # Number of bivariate basis
    LNB = LNU + int(N*(N-1)*M*(M-1)/4)

    indices = chaospy.glexindex(M+1, dimensions = N)

    inU = np.array(indices[0:LNU,:])
    inB = np.array(indices[0:(LNB),:])
    #inU = []
    cntU = 0
    #Multi-index for univariate (inU)
    for ii in range(LNM):
        cnt = 0
        for jj in range(N):
            if indices[ii,jj] != 0: 
                cnt += 1 
        if cnt == 1: # Univariate
            cntU += 1  
            inU[cntU,:] = indices[ii,:]
        
    cntB, cntU = 0, 0
    ii = 0
    jj = 0
    #Multi-index for bivariate (inB)
    for ii in range(LNM):
        cnt = 0
        for jj in range(N):
            if indices[ii,jj] != 0:
                cnt += 1
        if cnt == 1:
            cntB += 1
            inB[cntB,:] = indices[ii,:]
        if cnt == 2: # Bivariate
            cntB += 1
            inB[cntB,:] = indices[ii,:]

    # Generate linear system for regression
    # Information matrix
    # Initialize information matrix(inM):
    nSample1 = len(samples)
    inM = np.ones((nSample1, LNB))

    for i in range(nSample1):
        print(f"Q Progress: {i + 1}/{nSample1}         ", end="\r")
        for j in range(LNB):
            tmp = 1
            for k in range(N):
                crt = int(inB[j][k])
                tmp *= np.polynomial.polynomial.polyval(samples_transformed[i,k], fcoefficients_Q[k][crt])
                if crt != 0:
                    inM[i][j] = tmp

    return model.predict(inM)

# Given: cdf value,
# Generate: truncated normal random variable
# Note the transformation that is required: 
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.truncnorm.html
def truncated_normal(mean, std_dev, lower_bound, upper_bound, cdfval):
    a = (lower_bound - mean) / std_dev
    b = (upper_bound - mean) / std_dev
    return truncnorm.ppf(cdfval, a, b, loc=mean, scale=std_dev)

def ls_reg(inM,b): 
# Least squares to estimate unknown expansion coefficients 
    inMT = inM.T
    inM2 = np.dot(inMT,inM)
    inM3 = np.linalg.inv(inM2)
    inM4 = np.dot(inM3, inMT)
    sC = inM4.dot(b)
    return sC

def E2(poly, dist=None, **kws):
    if dist is None:
        dist, poly = poly, numpoly.variable(len(poly))

    poly = numpoly.set_dimensions(poly, len(dist))
    if poly.isconstant():
        return poly.tonumpy()

    moments = dist.mom(poly.exponents.T, **kws)
    if len(dist) == 1:
        moments = moments[0]

    out = np.zeros(poly.shape)
    for idx, key in enumerate(poly.keys):
        out += poly.values[key] * moments[idx]
    return out

def mat_inverse(mat):
    row = len(mat)
    col = len(mat[0])
    if row < col:
        tmp1 = np.dot(mat,mat.T)
        tmp2 = np.dot(mat.T, np.linalg.inv(tmp1))
    if row > col:
        tmp1 = np.dot(mat.T, mat)
        tmp2 = np.dot(np.linalg.inv(tmp1),mat.T)
    return tmp2 

def d_reg(Uff,Vff,i_sol):
    sol1 = np.linalg.inv(np.dot(Uff.T, Vff))
    sol2 = np.dot(Vff,sol1)
    sol3 = np.dot(sol2,Uff.T)
    sol = np.dot(sol3,np.array(i_sol))
    return sol

def d_reg2(model,Uff,Vff,Uff2,Vff2,Tr,i_sol,fproj):
# First term
    sol1 = d_reg(Uff,Vff,i_sol)
# Second term
    sol2_1 = np.linalg.inv(np.dot(Uff2.T, Vff2))
    sol2_2 = np.dot(Vff2,sol2_1)
    sol2_3 = np.dot(sol2_2, Tr) 
    sol2_4 = np.dot(sol2_3, Uff2.T)
    sol2_5 = np.dot(sol2_4, fproj)
    cd = np.concatenate(([model.intercept_], model.coef_[1:]))
    sol2 = sol1 + np.dot(sol2_5, cd)
    return sol2


def d_reg4(model,solf,Uff,Vff,Uff2,Vff2,Tr,i_sol,fproj,lamd):
# First term
    sol1 = d_reg(Uff,Vff,i_sol)
# Second term
    sol2_1 = np.linalg.inv(np.dot(Uff2.T, Vff2))
    sol2_2 = np.dot(Vff2,sol2_1)
    sol2_3 = np.dot(sol2_2, Tr) 
    sol2_4 = np.dot(sol2_3, Uff2.T)
    sol2_5 = np.dot(sol2_4, fproj)
    cd = np.concatenate(([model.intercept_], model.coef_[1:]))
    solf = cd*lamd + (1-lamd)*solf
    sol2 = sol1 + np.dot(sol2_5, solf)
    return sol2

def dmorph_f(inM1,bc):
    model = LassoLarsCV(cv=20,max_iter=1000)
    model.fit(inM1,bc)
    cd = np.concatenate(([model.intercept_], model.coef_[1:]))

    prj = mat_inverse(inM1)
    iden_mat = np.identity(len(prj))
    orth_prj = iden_mat - np.dot(prj,inM1) 

    i_sol = np.dot(prj,bc)

    iden_matD = np.identity(len(orth_prj))
    
    fproj = np.dot(orth_prj,iden_matD)
    # Singular value decomposition 
    Uf, Sf, Vf = scipy.linalg.svd(fproj)
    cnt = 0
    for i in range(len(Sf)):
        cnt += 1
        if abs(Sf[i]) < 1e-6:
            break
    cnt = cnt - 1
    Tr = np.diag(1/Sf[:cnt-1])
    Uff = Uf[:,cnt:]
    Vff = Vf.T[:,cnt:]
    Uff2 = Uf[:,:(cnt-1)]
    Vff2 = Vf.T[:,:(cnt-1)]
    sol1 = d_reg(Uff,Vff,i_sol)

    sol2 = d_reg2(model,Uff,Vff,Uff2,Vff2,Tr,i_sol,fproj)
    return sol2, model

def dmorph_s(model,inM1,bc,solf,lamd, iter2):
    # small value
    tol = 1e-5
    # Iterative scheme
    crt = 0
    solfvec = []
    prj = mat_inverse(inM1)
    iden_mat = np.identity(len(prj))
    orth_prj = iden_mat - np.dot(prj,inM1) 
    i_sol = np.dot(prj,bc)
    cd = np.concatenate(([model.intercept_], model.coef_[1:]))
    while crt < iter2:
        crt += 1
        print(f"Progress: {crt}/{iter2}", end="\r")
        iden_matD = np.identity(len(orth_prj))
        for i in range(len(iden_matD)):
            iden_matD[i][i] = (1/(abs(solf[i]) + tol))
        iden_matD[0][0] = 0
        fproj = np.dot(orth_prj,iden_matD)
        # Singular value decomposition 
        Uf, Sf, Vf = scipy.linalg.svd(fproj)
        cnt = 0
        for i in range(len(Sf)):
            cnt += 1
            if abs(Sf[i]) < 1e-5:
                break
        cnt = cnt - 1
        Tr = np.diag(1/Sf[:cnt-1])
        Uff = Uf[:,cnt:]
        Vff = Vf.T[:,cnt:]
        Uff2 = Uf[:,:(cnt-1)]
        Vff2 = Vf.T[:,:(cnt-1)] 
        solf = d_reg4(model,solf,Uff,Vff,Uff2,Vff2,Tr,i_sol,fproj,lamd)
        solfvec.append((solf))
        solf = (sum(solfvec[:])/crt)
    return solf

def obj_grad_fun(dv):
    global initial_guess, Samples, infoM_iter_1, nA, cy_iter_1,  N1, N2, N, Nd, M, fcoefficients

    # Calculate cy --------------
   
    # Information matrix: polynomial bases for Q
    infoM, _, _ = Cal_infoM(samples = Samples, \
                            N = N, M = M, \
                            dv = dv, \
                            dv_0 = initial_guess, \
                             fcoefficients = fcoefficients)
    # Single step GPCE: u2se MFiX's cy (cy_iter_1) to approximate Q
    Q = infoM.dot(cy_iter_1)

    # D-MORPH for cy
    temp_solf, temp_mdl = dmorph_f(infoM_iter_1, Q)
    cy = dmorph_s(model = temp_mdl, inM1 = infoM_iter_1, bc = Q, solf = temp_solf, \
                  lamd = .2, iter2 = 6)

    # Objective function value - based on coefficients of GPCE for Q ------------
    meanQ_LS = cy[0]
    varQ_LS = sum((cy[1:])**2)
    stdQ_LS = varQ_LS**0.5

    print(meanQ_LS)
    objVal = 0.5 * N1/meanQ_LS + 0.5 * stdQ_LS/N2               
    return objVal

In [ ]:
#############################################################
# Read input and opoutput data, 
# calculate the information matrix and indices for GPCE
#############################################################
# Generate indices
N = 5 # Number of RVs
M = 11 # Max order of GPCE bases

# RV1
a1, b1 = 0.10, 0.14
df = {'1': chaospy.Uniform(a1, b1)}
# print(exp1)
# RV2
a2, b2, mu2, sigma2 = 0.425, 1.225, 0.825, 0.0825
df['2'] = chaospy.TruncNormal(a2, b2, mu2, sigma2)
# print(exp2)
# RV3
a3, b3 = 2e-4, 1.4e-3
df['3'] =  chaospy.Uniform(a3, b3)
# print(exp3)
# RV4
a4, b4 = 0.5e-4, 1.5e-3
df['4'] = chaospy.Uniform(a4, b4)
# print(exp4)
# RV5
a5, b5, mu5, sigma5 = 1.35e-6, 13.5e-6, 7.35e-6, 7.35e-7
df['5'] = chaospy.TruncNormal(a5, b5, mu5, sigma5)

# open the CSV file in read mode
with open('samples.csv', mode = 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    samples = []
    for row in csv_reader:
        row = [float(x) for x in row]
        samples.append(row)
        
with open('samples2.csv', mode = 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    samples2 = []
    for row in csv_reader:
        row = [float(x) for x in row]
        samples2.append(row)
#        
samples = np.vstack((samples[0:100], samples2[100:200]))
with open('QoI.csv', mode = 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    QoI = []
    for row in csv_reader:
        row = [float(x) for x in row]
        QoI.append(row)

with open('QoI6.csv', mode = 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    QoI3 = []
    for row in csv_reader:
        row = [float(x) for x in row]
        QoI3.append(row)

QoI2 = np.hstack((QoI, QoI3))

inM, LNB, inB = PDD_approx_ex2(N=N, M=M, S=2,df=df,samples=samples)

exct_mean = cal_average(QoI2[0])
exct_var = (cal_variance(QoI2[0]))
print(exct_mean)
print(exct_var**0.5)

1414.6351985052163
195.20486307774402


In [ ]:
#############################################################
# LASSO REGRESSION
#############################################################

ind = int(len(inM))
cut  = 200
las_mean = []
las_std = []

nT = 1

In [31]:
%time
lamd = .2
niter = 6
for i in range(nT):
    print('iter =', i+1)
    np.random.seed(1234+i)
    S = np.random.permutation(ind)
    inM2 = inM[S[:cut]]
    bc = np.array(QoI2[0][S[:cut]])
    sol2, model = dmorph_f(inM2,bc)
    solf = dmorph_s(model,inM2,bc,sol2,lamd,niter)
    dm_var_app = sum((solf[1:])**2)
    dm_std_app = dm_var_app**0.5
    print(solf[0])
    print(dm_std_app)
print(solf)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
iter = 1
1436.4472962686834
198.81439739464415
[ 1.43644730e+03 -3.63172493e+00  1.40574025e+00  8.61250851e+00
  1.73801481e+00  1.25802579e+00  3.03754019e+00  7.86233306e-01
 -1.40534126e+00 -1.03760120e-01  7.22662720e+00  3.02018217e+00
  1.46519322e+02  1.04985304e+00  6.59679697e-01  8.46463818e-01
 -1.76750500e+00 -1.61532608e+00  1.18049992e+00  5.88515856e-02
 -6.69614425e-01  7.62615963e-01 -1.12453566e+00  1.21046008e+00
 -1.10360929e+00  2.32284691e+00 -7.56299424e-02  1.51106561e+00
 -6.02336238e-01 -1.56062575e+00 -7.16731868e-01 -4.14763252e-01
  1.49166251e+00 -2.32939881e+00 -2.36921650e+00 -2.89424649e+00
  1.37785284e+00  2.79416921e-01  1.11372439e+00 -1.29583025e+00
 -6.15721126e-01  4.45831792e+00 -7.62336867e-01  1.58143910e+00
 -2.01257606e+00 -1.96544402e+00 -4.94053523e-01  5.67198058e-01
 -5.85052635e-01  2.04336980e+00  2.10525552e+00  1.44915913e+00
  2.76819016e+00 -1.95754683e+00  1.68766929

## Check the accuracy of PDD surrogate

In [24]:
from sklearn.metrics import r2_score

In [28]:
Q_LASSO = inM.dot(np.concatenate(([model.intercept_], model.coef_[1:])))
Q_PDD = inM.dot(solf)

In [29]:
print(r2_score(QoI2[0, :], Q_LASSO))
print(r2_score(QoI2[0, :], Q_PDD))

0.976723277570554
0.9967438837286472


### PDD: coefficents, information matrix, etc

In [33]:
fcoefficients = Generate_expansion(M)
fcoefficients_Q = Generate_expansion_Q(M , df = df)

## Optimization

Some variables calculated in iteration 1 that will be used in all iterations:<br>
<br>
initial_guess<br>
Samples (equivalent to 'z' in the RDO paper)<br>
infoM_iter_1 (infomation matrix calculated in first evaluation of objective function, using Samples)<br>
nA (number of uni- and bi-variate bases)<br>
cy_iter_1<br>
cs_iter_1<br>
tri_ON<br>
N1 (normalizing factor for the first term in the objective function)<br>
N2 (normalizing factor, second term)<br>
fcoefficients (expansion coefficients)

In [ ]:
# cy ------------
cy_iter_1 = solf

In [35]:
# Normalizing factor
N1, N2 = cy_iter_1[0], sum((cy_iter_1[1:])**2)**0.5
print(N1)
print(N2)

1436.4472962686834
198.81439739464415


In [ ]:
# Samples -----------------
# Scaling transformation:
sampler = scipy.stats.qmc.Sobol(d = 5, seed = np.random.default_rng(4321))
# sampler = scipy.stats.qmc.Sobol(d = N)
sample = sampler.random(200)
# Samples for output y
Samples = np.empty((200, 5))
# equivalent to z in the RDO paper
# Z1
lb_1, ub_1 = 0.1, 0.14
Samples[:, 0] = ((ub_1 - lb_1) * sample[:, 0] + lb_1)
# Z2
mu_2 = 0.825
lb_2 = 0.425
ub_2 = 1.225
Samples[:, 1] = truncated_normal(1, 0.1, lb_2/mu_2, ub_2/mu_2, sample[:, 1])
# Z3
mu_3 = 8e-4
lb_3, ub_3 = 2e-4, 1.4e-3
Samples[:, 2] = (ub_3/mu_3 - lb_3/mu_3) * sample[:, 2] + lb_3/mu_3
# Z4
lb_4, ub_4 = 0.5e-4, 1.5e-3
Samples[:, 3] = ((ub_4 - lb_4) * sample[:, 3] + lb_4)
# Z5
Samples[:, 4] = (truncated_normal(7.35e-6, 7.35e-7, 1.35e-6, 1.35e-5, sample[:, 4]))

# Calculate these at the first iteration:
# Information matrix, ...
# multi-index for uni and bivariate terms in the expansion, ...
# Number of uni and bivariate basis, and ...
# Thermal energy
initial_guess = [0.825, 0.8e-3]
Nd = len(initial_guess)
infoM_iter_1, _, _ = Cal_infoM(samples = Samples, \
                               N = N, M = M, \
                               dv = initial_guess, \
                               dv_0 = initial_guess, \
                               fcoefficients = fcoefficients)
Q_Samples = Cal_Q(samples = Samples, N = N, M = M, dv = initial_guess, fcoefficients_Q = fcoefficients_Q)

In [37]:
%time
Cal_Q(samples = Samples, N = N, M = M, dv = initial_guess, fcoefficients_Q = fcoefficients_Q)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 7.15 µs


array([1845.69342182, 1238.96742787, 1723.38830093, 1324.74624523,
       1349.64321925, 1329.58773874, 1515.86243111, 1452.10359872,
       1435.95270729, 1639.14286776, 1524.32263215, 1258.15701295,
       1657.53388728, 1140.80389807, 1440.57999691, 1320.30582975,
       1705.69996198, 1301.23852422, 1514.58870304, 1054.06157977,
       1566.14099558, 1283.63729645, 1405.56533833, 1428.07476428,
       1482.04116013, 1276.59272493, 1362.17440392, 1495.08007225,
       1491.46551566, 1333.98368984, 1894.2438291 , 1238.94032596,
       1390.78807156, 1333.85383177, 1695.60083209, 1558.04218334,
       1637.16276304, 1165.87892133, 1556.40208463, 1283.39813435,
       1800.36144278, 1238.6708578 , 1550.73973489, 1339.52564171,
       1363.48209083, 1628.00725572, 1447.63447257, 1151.4905704 ,
       1514.11389402, 1137.99461497, 1389.41892359, 1371.42693187,
       1690.24709267, 1325.84132758, 1657.1994927 , 1204.39267069,
       1443.09702876, 1273.22774003, 1861.29016901, 1182.58062

# Optimization run 

In [14]:
history = {'x': [], 'fval': []}
searchdir = []
Nfeval = 1

def callback(xk):
    global Nfeval
    fval = obj_grad_fun(xk)
    history['x'].append(xk)
    history['fval'].append(fval)
    print('{0:4d}   {1: 3.6f}   {2: 3.6f}   {3: 3.6f}'.format(Nfeval, xk[0], xk[1], fval))
    Nfeval += 1

print('{0:4s}   {1:9s}   {2:9s}   {3:9s}'.format('Iter', ' d1', ' d2', 'f(X)')) 
    
options = {'disp': True, 'fatol': 7e-1, 'xatol': 7e-3, 'maxiter': 8} 
bnd = [[0.625, 1.025], [5e-4, 1.1e-3]]

# minimize
res = minimize(obj_grad_fun, x0= initial_guess,  method='Nelder-Mead', bounds=bnd, 
                   options=options,  callback=callback)  # !!!
print(res.x)

Iter    d1          d2         f(X)     
1438.13747680159551/200
1509.33996651277601/200
1417.26823629523821/200
1515.34578255157601/200
1493.99586946887051/200
1438.13747680159551/200
   1    0.825000    0.000800    0.988687
1451.98795629443161/200
1438.13747680159551/200
   2    0.825000    0.000800    0.988687
1380.31667240420731/200
1311.20008720608101/200
1380.31667240420731/200
   3    0.794062    0.000820    0.984369
1364.99133379992171/200
1319.55630495647441/200
1319.55630495647441/200
   4    0.757969    0.000790    0.980570
1262.82373022502931/200
1319.55630495647441/200
   5    0.757969    0.000790    0.980570
1202.83458126908571/200
1335.59936638445801/200
1335.59936638445801/200
   6    0.768281    0.000810    0.977097
1393.49012549011061/200
1360.76882725425431/200
1335.59936638445801/200
   7    0.768281    0.000810    0.977097
[0.76828125 0.00081   ]
